In [1]:
!pip install faiss-cpu
import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 80.7 MB/s eta 0:00:00


In [2]:
DATA_FOLDER = "data"
EMBED_MODEL = "all-MiniLM-L6-v2"
TOP_K = 3
MODEL_NAME = "gpt-4o-mini"

In [6]:
from getpass import getpass
from openai import OpenAI

# Ask user for OpenAI API key at runtime (hidden input)
OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")

# Create OpenAI client using the user-provided key
client = OpenAI(api_key=OPENAI_API_KEY)

# Your embedding model (remains the same)
embedder = SentenceTransformer(EMBED_MODEL)


Enter your OpenAI API key: ··········


In [7]:


client = OpenAI(api_key=OPENAI_API_KEY)
OPENAI_MODEL = "gpt-4o-mini"

In [ ]:
def load_documents(folder):
    docs = []
    for file in os.listdir(folder):
        if file.endswith(".txt"):
            with open(os.path.join(folder, file), "r", encoding="utf-8") as f:
                docs.append(f.read())
    return docs

In [ ]:
def chunk_text(text, chunk_size=300):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

documents = load_documents(DATA_FOLDER)
chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc))

print(f"✅ Loaded {len(chunks)} text chunks.")

print("🔢 Creating embeddings and FAISS index...")
doc_embeddings = embedder.encode(chunks)
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings).astype("float32"))
print(f"✅ FAISS index ready with {index.ntotal} vectors.\n")

✅ Loaded 4 text chunks.
🔢 Creating embeddings and FAISS index...
✅ FAISS index ready with 4 vectors.



In [ ]:
while True:
    query = input("🧠 Your question (or type 'quit'): ")
    if query.lower() == "quit":
        break
    query_embedding = embedder.encode([query])


    distances, indices = index.search(np.array(query_embedding).astype("float32"), TOP_K)
    retrieved_chunks = [chunks[i] for i in indices[0]]
    context = "\n".join(retrieved_chunks)

    prompt = f"""You are a helpful assistant.
Use the following context to answer the user's question.
If the answer isn't in the context, say "I don't know based on the provided documents."

Context:
{context}

Question: {query}
Answer:"""


    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )

    answer = response.choices[0].message.content.strip()
    print("\n💬 Answer:", answer, "\n" + "-"*80 + "\n")

🧠 Your question (or type 'quit'): what are the types in machine learning

💬 Answer: The three main types of machine learning are supervised learning, unsupervised learning, and reinforcement learning. 
--------------------------------------------------------------------------------



KeyboardInterrupt: Interrupted by user